In [ ]:

        # add a point of interpolate object annotation
        self.sam_interpolate_annotation_button = QtWidgets.QPushButton()
        self.sam_interpolate_annotation_button.setStyleSheet(
            "QPushButton { font-size: 10pt; font-weight: bold; }")
        self.sam_interpolate_annotation_button.setText("Interpolate Annotation")
        self.sam_interpolate_annotation_button.clicked.connect(
            self.sam_interpolate_annotation_button_clicked)
        self.sam_toolbar.addWidget(self.sam_interpolate_annotation_button)
        

    def sam_interpolate_annotation_button_clicked(self):
        x = self.INDEX_OF_CURRENT_FRAME
        self.sam_model_comboBox.setCurrentIndex(1)
        max_id = 0
        for i in range(1, x + 1, 10):
            self.main_video_frames_slider.setValue(i)
            self.main_video_frames_slider_changed()
            try:
                same_image = self.sam_predictor.check_image(
                    self.CURRENT_FRAME_IMAGE)
            except:
                return
            if not same_image:
                self.sam_clear_annotation_button_clicked()
                self.sam_buttons_colors("X")
            print(f'------------------------------------- now in frame {self.INDEX_OF_CURRENT_FRAME}-------------------------------------')
            for shapei in range(len(self.CURRENT_SHAPES_IN_IMG)):
                shape = self.CURRENT_SHAPES_IN_IMG[shapei]
                print(f'agjusting points of shape with id = {self.CURRENT_SHAPES_IN_IMG[shapei]["group_id"]}')
                x1, y1, x2, y2 = shape["bbox"]
                listPOINTS = [min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)]
                listPOINTS = [int(round(x)) for x in listPOINTS]
                input_boxes = [listPOINTS]
                mask, score = self.sam_predictor.predict(point_coords=None,
                                                 point_labels=None,
                                                 box=input_boxes,
                                                 image=self.CURRENT_FRAME_IMAGE)
                points = self.sam_predictor.mask_to_polygons(mask)
                shape = self.sam_predictor.polygon_to_shape(points, score)
                self.CURRENT_SHAPES_IN_IMG[shapei]["points"] = shape["points"]
                id = self.CURRENT_SHAPES_IN_IMG[shapei]["group_id"]
                max_id = max(max_id, id)
                try:
                    self.key_frames['id_' +
                                    str(id)].append(self.INDEX_OF_CURRENT_FRAME)
                except:
                    self.key_frames['id_' +
                                str(id)] = [self.INDEX_OF_CURRENT_FRAME]
            print(f'updating shapes the updating current frame annotation')
            self.canvas.shapes = convert_shapes_to_qt_shapes(self.CURRENT_SHAPES_IN_IMG)
            self.update_current_frame_annotation()
            self.main_video_frames_slider_changed()
        
        for i in range(1, max_id + 1):
            print(f'interpolating id = {i}')
            try:
                x = self.key_frames['id_' +
                                    str(id)]
            except:
                continue
            self.interpolate(id=i, only_edited=True)


